In [1]:
import torch

if torch.cuda.is_available():
    print(f"✅ Succès ! GPU détecté : {torch.cuda.get_device_name(0)}")
    device = torch.device("cuda")
else:
    print("⚠️ Attention : Aucun GPU détecté, vous êtes sur CPU (ça va être lent).")
    device = torch.device("cpu")

# Note sur les TPU :
# Sur Colab, pour PyTorch, on évite généralement les TPU car cela demande 
# une librairie spéciale (torch_xla). Restez sur GPU ("T4") pour votre projet.

⚠️ Attention : Aucun GPU détecté, vous êtes sur CPU (ça va être lent).
